In [ ]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `qwiklabs-gcp-02-3205ce60aeba.fraud_data`;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
DROP TABLE IF EXISTS `qwiklabs-gcp-02-3205ce60aeba.fraud_data.fraud_training_data`;
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-02-3205ce60aeba.fraud_data.fraud_training_data`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv'],
  skip_leading_rows = 1
);
SELECT
  *,
  -- Employment_Status one-hot encoding
  CASE WHEN Employment_Status = 'Employed' THEN 1 ELSE 0 END AS is_employed,
  CASE WHEN Employment_Status = 'Self-employed' THEN 1 ELSE 0 END AS is_self_employed,
  CASE WHEN Employment_Status = 'Unemployed' THEN 1 ELSE 0 END AS is_unemployed,
  -- Device_Type one-hot encoding
  CASE WHEN Device_Type = 'Desktop' THEN 1 ELSE 0 END AS is_desktop,
  CASE WHEN Device_Type = 'Mobile' THEN 1 ELSE 0 END AS is_mobile,
  CASE WHEN Device_Type = 'Tablet' THEN 1 ELSE 0 END AS is_tablet,

-- Age bin one-hot encoding
  CASE WHEN Age BETWEEN 18 AND 24 THEN 1 ELSE 0 END AS age_18_24,
  CASE WHEN Age BETWEEN 25 AND 34 THEN 1 ELSE 0 END AS age_25_34,
  CASE WHEN Age BETWEEN 35 AND 44 THEN 1 ELSE 0 END AS age_35_44,
  CASE WHEN Age BETWEEN 45 AND 54 THEN 1 ELSE 0 END AS age_45_54,
  CASE WHEN Age >= 55 THEN 1 ELSE 0 END AS age_55_plus,


-- Income-to-Amount-Requested ratio
SAFE_DIVIDE(Income, Amount_Requested) AS Income_to_Amount_Requested,

DATE_DIFF(CURRENT_DATE(), DATE(Previous_Assistance_Date), DAY) AS Time_Since_Previous_Assistance,


-- Boolean to binary
CAST(Previous_Assistance_Received AS INT64) AS previous_assistance_received_binary,
CAST(Supporting_Doc_Verified AS INT64) AS supporting_doc_verified_binary


FROM
  `fraud_data.fraud_training_data`;

Query is running:   0%|          |

Downloading:   0%|          |

,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,...,is_tablet,age_18_24,age_25_34,age_35_44,age_45_54,age_55_plus,Income_to_Amount_Requested,Time_Since_Previous_Assistance,previous_assistance_received_binary,supporting_doc_verified_binary
0,212,18,Employed,0,3,6987,False,NaT,True,10,...,0,1,0,0,0,0,0.000000,<NA>,0,1
1,262,18,Employed,71639,3,937,False,NaT,False,3,...,0,1,0,0,0,0,76.455710,<NA>,0,0
2,423,18,Self-Employed,0,2,7358,False,NaT,True,1,...,0,1,0,0,0,0,0.000000,<NA>,0,1
3,480,18,Employed,0,2,1205,True,2024-06-17,False,7,...,0,1,0,0,0,0,0.000000,444,1,0
4,486,18,Employed,0,5,2256,False,NaT,True,2,...,0,1,0,0,0,0,0.000000,<NA>,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49969,65,Employed,60106,3,6387,True,2022-11-19,True,3,...,1,0,0,0,0,1,9.410678,1020,1,1
49996,49972,65,Employed,58000,3,6258,True,2023-11-09,False,5,...,1,0,0,0,0,1,9.268137,665,1,0
49997,49982,65,Unemployed,62766,2,8207,False,NaT,False,9,...,0,0,0,0,0,1,7.647862,<NA>,0,0
49998,50124,65,Self-Employed,0,4,7604,True,2024-01-30,True,1,...,1,0,0,0,0,1,0.000000,583,1,1
